In [1]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install -q kaggle
!pip install pillow
!kaggle datasets download -d briscdataset/brisc2025
!unzip -q brisc2025.zip -d ./archive

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/briscdataset/brisc2025
License(s): Attribution 4.0 International (CC BY 4.0)
 62% 155M/250M [00:00<00:00, 1.62GB/s]
100% 250M/250M [00:00<00:00, 998MB/s] 


In [2]:
import os
from PIL import Image
from glob import glob
import cv2
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [3]:
# Set paths

image_input_dir = "/content/archive/brisc2025/classification_task/train/glioma"          # folder containing original .jpg images
mask_input_dir = "/content/archive/brisc2025/classification_task/train/pituitary"            # folder containing original .png masks

image_output_dir = "/content/archive/brisc2025/classification_input/glioma" # folder to save resized images
mask_output_dir = "/content/archive/brisc2025/classification_input/pituitary"   # folder to save resized masks

# Desired output size
new_size = (256, 256)   # (width, height)

# Create output directories if they don't exist
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(mask_output_dir, exist_ok=True)


def resize_image(input_path, output_path, size, is_mask=False):
    img = Image.open(input_path)

    # For masks → nearest neighbor (keeps integer class values)
    if is_mask:
        img = img.resize(size, Image.NEAREST)
    else:
        # For images → bilinear interpolation
        img = img.resize(size, Image.BILINEAR)

    img.save(output_path)


# Resize all JPG images
for filename in os.listdir(image_input_dir):
    if filename.lower().endswith(".jpg"):
        input_path = os.path.join(image_input_dir, filename)
        output_path = os.path.join(image_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=False)
        print("Resized image:", filename)

# Resize all PNG masks
for filename in os.listdir(mask_input_dir):
    if filename.lower().endswith(".png"):
        input_path = os.path.join(mask_input_dir, filename)
        output_path = os.path.join(mask_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=True)
        print("Resized mask:", filename)

print("Done! Images and masks resized and saved.")

Resized image: brisc2025_train_01085_gl_sa_t1.jpg
Resized image: brisc2025_train_00067_gl_ax_t1.jpg
Resized image: brisc2025_train_00918_gl_sa_t1.jpg
Resized image: brisc2025_train_00477_gl_co_t1.jpg
Resized image: brisc2025_train_00930_gl_sa_t1.jpg
Resized image: brisc2025_train_01044_gl_sa_t1.jpg
Resized image: brisc2025_train_01140_gl_sa_t1.jpg
Resized image: brisc2025_train_00808_gl_co_t1.jpg
Resized image: brisc2025_train_00455_gl_co_t1.jpg
Resized image: brisc2025_train_01114_gl_sa_t1.jpg
Resized image: brisc2025_train_00009_gl_ax_t1.jpg
Resized image: brisc2025_train_00613_gl_co_t1.jpg
Resized image: brisc2025_train_00272_gl_ax_t1.jpg
Resized image: brisc2025_train_00764_gl_co_t1.jpg
Resized image: brisc2025_train_00924_gl_sa_t1.jpg
Resized image: brisc2025_train_00972_gl_sa_t1.jpg
Resized image: brisc2025_train_00645_gl_co_t1.jpg
Resized image: brisc2025_train_00525_gl_co_t1.jpg
Resized image: brisc2025_train_00249_gl_ax_t1.jpg
Resized image: brisc2025_train_00528_gl_co_t1.jpg


In [4]:
# Set paths

image_input_dir = "/content/archive/brisc2025/classification_task/train/pituitary"          # folder containing original .jpg images
mask_input_dir = "/content/archive/brisc2025/classification_task/train/pituitary1"            # folder containing original .png masks

image_output_dir = "/content/archive/brisc2025/classification_input/pituitary" # folder to save resized images
mask_output_dir = "/content/archive/brisc2025/classification_input/pituitary1"   # folder to save resized masks

# Desired output size
new_size = (256, 256)   # (width, height)

# Create output directories if they don't exist
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(mask_output_dir, exist_ok=True)


def resize_image(input_path, output_path, size, is_mask=False):
    img = Image.open(input_path)

    # For masks → nearest neighbor (keeps integer class values)
    if is_mask:
        img = img.resize(size, Image.NEAREST)
    else:
        # For images → bilinear interpolation
        img = img.resize(size, Image.BILINEAR)

    img.save(output_path)


# Resize all JPG images
for filename in os.listdir(image_input_dir):
    if filename.lower().endswith(".jpg"):
        input_path = os.path.join(image_input_dir, filename)
        output_path = os.path.join(image_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=False)
        print("Resized image:", filename)

# Resize all PNG masks
for filename in os.listdir(mask_input_dir):
    if filename.lower().endswith(".png"):
        input_path = os.path.join(mask_input_dir, filename)
        output_path = os.path.join(mask_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=True)
        print("Resized mask:", filename)

print("Done! Images and masks resized and saved.")

Resized image: brisc2025_train_04342_pi_co_t1.jpg
Resized image: brisc2025_train_04997_pi_sa_t1.jpg
Resized image: brisc2025_train_03744_pi_ax_t1.jpg
Resized image: brisc2025_train_03665_pi_ax_t1.jpg
Resized image: brisc2025_train_03730_pi_ax_t1.jpg
Resized image: brisc2025_train_04751_pi_sa_t1.jpg
Resized image: brisc2025_train_04443_pi_co_t1.jpg
Resized image: brisc2025_train_04729_pi_sa_t1.jpg
Resized image: brisc2025_train_04340_pi_co_t1.jpg
Resized image: brisc2025_train_03868_pi_ax_t1.jpg
Resized image: brisc2025_train_03674_pi_ax_t1.jpg
Resized image: brisc2025_train_03895_pi_ax_t1.jpg
Resized image: brisc2025_train_03859_pi_ax_t1.jpg
Resized image: brisc2025_train_03834_pi_ax_t1.jpg
Resized image: brisc2025_train_04506_pi_sa_t1.jpg
Resized image: brisc2025_train_04516_pi_sa_t1.jpg
Resized image: brisc2025_train_04380_pi_co_t1.jpg
Resized image: brisc2025_train_04024_pi_co_t1.jpg
Resized image: brisc2025_train_04956_pi_sa_t1.jpg
Resized image: brisc2025_train_04402_pi_co_t1.jpg


FileNotFoundError: [Errno 2] No such file or directory: '/content/archive/brisc2025/classification_task/train/pituitary1'

In [5]:
# Set paths

image_input_dir = "/content/archive/brisc2025/classification_task/train/meningioma"          # folder containing original .jpg images
mask_input_dir = "/content/archive/brisc2025/classification_task/train/meningioma1"            # folder containing original .png masks

image_output_dir = "/content/archive/brisc2025/classification_input/meningioma" # folder to save resized images
mask_output_dir = "/content/archive/brisc2025/classification_input/meningioma1"   # folder to save resized masks

# Desired output size
new_size = (256, 256)   # (width, height)

# Create output directories if they don't exist
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(mask_output_dir, exist_ok=True)


def resize_image(input_path, output_path, size, is_mask=False):
    img = Image.open(input_path)

    # For masks → nearest neighbor (keeps integer class values)
    if is_mask:
        img = img.resize(size, Image.NEAREST)
    else:
        # For images → bilinear interpolation
        img = img.resize(size, Image.BILINEAR)

    img.save(output_path)


# Resize all JPG images
for filename in os.listdir(image_input_dir):
    if filename.lower().endswith(".jpg"):
        input_path = os.path.join(image_input_dir, filename)
        output_path = os.path.join(image_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=False)
        print("Resized image:", filename)

# Resize all PNG masks
for filename in os.listdir(mask_input_dir):
    if filename.lower().endswith(".png"):
        input_path = os.path.join(mask_input_dir, filename)
        output_path = os.path.join(mask_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=True)
        print("Resized mask:", filename)

print("Done! Images and masks resized and saved.")

Resized image: brisc2025_train_01676_me_co_t1.jpg
Resized image: brisc2025_train_01501_me_ax_t1.jpg
Resized image: brisc2025_train_01783_me_co_t1.jpg
Resized image: brisc2025_train_02365_me_sa_t1.jpg
Resized image: brisc2025_train_01926_me_co_t1.jpg
Resized image: brisc2025_train_01800_me_co_t1.jpg
Resized image: brisc2025_train_01812_me_co_t1.jpg
Resized image: brisc2025_train_01495_me_ax_t1.jpg
Resized image: brisc2025_train_01824_me_co_t1.jpg
Resized image: brisc2025_train_01725_me_co_t1.jpg
Resized image: brisc2025_train_01230_me_ax_t1.jpg
Resized image: brisc2025_train_01460_me_ax_t1.jpg
Resized image: brisc2025_train_01530_me_ax_t1.jpg
Resized image: brisc2025_train_02458_me_sa_t1.jpg
Resized image: brisc2025_train_01450_me_ax_t1.jpg
Resized image: brisc2025_train_01722_me_co_t1.jpg
Resized image: brisc2025_train_02317_me_sa_t1.jpg
Resized image: brisc2025_train_02145_me_sa_t1.jpg
Resized image: brisc2025_train_01253_me_ax_t1.jpg
Resized image: brisc2025_train_01381_me_ax_t1.jpg


FileNotFoundError: [Errno 2] No such file or directory: '/content/archive/brisc2025/classification_task/train/meningioma1'

In [6]:
# Set paths

image_input_dir = "/content/archive/brisc2025/classification_task/train/no_tumor"          # folder containing original .jpg images
mask_input_dir = "/content/archive/brisc2025/classification_task/train/no_tumor1"            # folder containing original .png masks

image_output_dir = "/content/archive/brisc2025/classification_input/no_tumor" # folder to save resized images
mask_output_dir = "/content/archive/brisc2025/classification_input/no_tumor1"   # folder to save resized masks

# Desired output size
new_size = (256, 256)   # (width, height)

# Create output directories if they don't exist
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(mask_output_dir, exist_ok=True)


def resize_image(input_path, output_path, size, is_mask=False):
    img = Image.open(input_path)

    # For masks → nearest neighbor (keeps integer class values)
    if is_mask:
        img = img.resize(size, Image.NEAREST)
    else:
        # For images → bilinear interpolation
        img = img.resize(size, Image.BILINEAR)

    img.save(output_path)


# Resize all JPG images
for filename in os.listdir(image_input_dir):
    if filename.lower().endswith(".jpg"):
        input_path = os.path.join(image_input_dir, filename)
        output_path = os.path.join(image_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=False)
        print("Resized image:", filename)

# Resize all PNG masks
for filename in os.listdir(mask_input_dir):
    if filename.lower().endswith(".png"):
        input_path = os.path.join(mask_input_dir, filename)
        output_path = os.path.join(mask_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=True)
        print("Resized mask:", filename)

print("Done! Images and masks resized and saved.")

Resized image: brisc2025_train_02675_no_ax_t1.jpg
Resized image: brisc2025_train_03033_no_co_t1.jpg
Resized image: brisc2025_train_03134_no_co_t1.jpg
Resized image: brisc2025_train_02818_no_ax_t1.jpg
Resized image: brisc2025_train_02816_no_ax_t1.jpg
Resized image: brisc2025_train_03204_no_sa_t1.jpg
Resized image: brisc2025_train_03125_no_co_t1.jpg
Resized image: brisc2025_train_02995_no_co_t1.jpg
Resized image: brisc2025_train_03256_no_sa_t1.jpg
Resized image: brisc2025_train_03355_no_sa_t1.jpg
Resized image: brisc2025_train_02709_no_ax_t1.jpg
Resized image: brisc2025_train_03007_no_co_t1.jpg
Resized image: brisc2025_train_03389_no_sa_t1.jpg
Resized image: brisc2025_train_03250_no_sa_t1.jpg
Resized image: brisc2025_train_02759_no_ax_t1.jpg
Resized image: brisc2025_train_03105_no_co_t1.jpg
Resized image: brisc2025_train_02583_no_ax_t1.jpg
Resized image: brisc2025_train_02728_no_ax_t1.jpg
Resized image: brisc2025_train_03502_no_sa_t1.jpg
Resized image: brisc2025_train_03146_no_sa_t1.jpg


FileNotFoundError: [Errno 2] No such file or directory: '/content/archive/brisc2025/classification_task/train/no_tumor1'

In [8]:
import os
from PIL import Image
from glob import glob
import cv2
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2


# ------------------ Custom Z-Score Normalization ------------------
class PerImageZScore(A.ImageOnlyTransform):
    def __init__(self, p=1.0):
        super().__init__(p=p)
    def apply(self, img, **params):
        img = img.astype(np.float32)
        m = img.mean()
        s = img.std()
        if s < 1e-6:
            s = 1e-6
        return (img - m) / s


#RESUNET MODEL CODE

class ConvBlock(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=3,padding=1,stride=2,bias=False):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False)
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1,bias=False)
        self.projectionConv = nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,padding=0, bias=False)
        self.relu = nn.ReLU(inplace = True)
        self.bn2=nn.BatchNorm2d(out_channels)
        self.bn1=nn.BatchNorm2d(in_channels)


    def forward(self,x):
        fx=self.bn1(x)
        fx = self.relu(fx)
        x = self.projectionConv(x)
        fx = self.conv1(fx)
        fx=self.bn2(fx)
        fx = self.relu(fx)
        fx = self.conv2(fx)
        return torch.add(x,fx)

class ConvBlock1(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=3,padding=1,stride=1,bias=False):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=1,padding=1,bias=False)
        self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1,bias=False)
        self.projectionConv = nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=1,padding=0, bias=False)
        self.relu = nn.ReLU(inplace = True)
        self.bn=nn.BatchNorm2d(out_channels)


    def forward(self,x):

        fx = self.conv1(x)
        x = self.projectionConv(x)
        fx=self.bn(fx)
        fx = self.relu(fx)
        fx = self.conv2(fx)
        return torch.add(x,fx)

class ConvBlockDec(nn.Module):
    def __init__(self,in_channels,out_channels):
        super().__init__()
        self.upconv = nn.ConvTranspose2d(in_channels,out_channels,kernel_size=2,stride=2, output_padding=0)
        self.ConvBlock = ConvBlock(2*out_channels,out_channels,stride=1)


    def forward(self,x,skip_connection):
        x = self.upconv(x)
        x = torch.cat((x, skip_connection), dim=1)
        x = self.ConvBlock(x)
        return x

class ResUnet(nn.Module):
    def __init__(self,in_channels=1,out_channels=1):
        super().__init__()
        self.enc1 = ConvBlock1(in_channels,64)
        self.enc2 = ConvBlock(64,128)
        self.enc3 = ConvBlock(128,256)
        self.enc4 = ConvBlock(256,512)

        self.base = ConvBlock(512,1024)

        self.dec4 = ConvBlockDec(1024,512)
        self.dec3 = ConvBlockDec(512,256)
        self.dec2 = ConvBlockDec(256,128)
        self.dec1 = ConvBlockDec(128,64)

        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1,bias = False)
        self._init_weights()

    def forward(self,x):
        s1 = self.enc1(x)
        s2 = self.enc2(s1)
        s3 = self.enc3(s2)
        s4 = self.enc4(s3)

        b = self.base(s4)

        d4 = self.dec4(b,s4)
        d3 = self.dec3(d4,s3)
        d2 = self.dec2(d3,s2)
        d1= self.dec1(d2,s1)

        y = self.final_conv(d1)

        return y


    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)


# ------------------ Grad-CAM ------------------
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()
        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0].detach()

        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_full_backward_hook(backward_hook)

    def __call__(self, x, mode="sum"):
        self.model.zero_grad()
        output = self.model(x)

        if mode == "center":
            target = output[0, 0, output.shape[2]//2, output.shape[3]//2]
        else:
            target = output.sum()

        target.backward(retain_graph=True)

        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        grad_cam_map = F.relu((weights * self.activations).sum(dim=1, keepdim=True))
        grad_cam_map = F.interpolate(grad_cam_map, size=x.shape[2:], mode='bilinear', align_corners=False)

        return grad_cam_map.squeeze().cpu().numpy(), output
# ------------------ Repro / Device ------------------
torch.manual_seed(42)
np.random.seed(42)
torch.backends.cudnn.benchmark = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# ------------------ Instantiate model and load checkpoint ------------------
model = ResUnet(in_channels=1, out_channels=1).to(device)
ckpt_path = '/content/BTresunet_model_1.2.pth'  # update path if needed
ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
try:
    model.load_state_dict(ckpt['model_state_dict'])
    print("Checkpoint loaded (strict=True).")
except Exception as e:
    print("Strict load failed:", e)
    # try forgiving load (handles 'module.' prefixes etc.)
    state_dict = ckpt['model_state_dict']
    new_state = {}
    for k, v in state_dict.items():
        new_k = k.replace('module.', '') if k.startswith('module.') else k
        new_state[new_k] = v
    model.load_state_dict(new_state, strict=False)
    print("Checkpoint loaded with strict=False (prefixes handled).")

model.eval()
# ------------------ Grad-CAM target layer ------------------
# final_conv is fine; for richer features consider e.g. model.enc3.conv2
target_layer = model.final_conv
grad_cam = GradCAM(model, target_layer)


Using device: cuda
Checkpoint loaded (strict=True).


In [9]:

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def save_mask_and_gradcam_from_dir(input_dir,
                                   mask_out_dir,
                                   grad_out_dir,
                                   model,
                                   gradcam,
                                   img_exts=('.jpg', '.jpeg', '.png'),
                                   threshold=0.5,
                                   resize_to=None):
    """
    Processes every image file in input_dir (matching img_exts).
    - saves segmentation probability map and binary mask to mask_out_dir
    - saves Grad-CAM heatmap (colored) to grad_out_dir
    - resize_to: (W,H) or None to keep original size
    """
    ensure_dir(mask_out_dir)
    ensure_dir(grad_out_dir)

    image_paths = []
    for ext in img_exts:
        image_paths += glob(os.path.join(input_dir, f'*{ext}'))
    image_paths = sorted(image_paths)
    if len(image_paths) == 0:
        print('No images found in', input_dir)
        return


    device = next(model.parameters()).device

    for p in image_paths:
        name = Path(p).name
        name_stem = Path(p).stem

        # --- load and preprocess similarly to your test code ---
        pil = Image.open(p).convert('L')
        img_np = np.array(pil).astype(np.float32) / 255.0  # H x W

        # optional resize here to the model's expected input if needed:
        if resize_to is not None:
            img_disp = cv2.resize((img_np * 255).astype(np.uint8), resize_to, interpolation=cv2.INTER_LINEAR)
            img_for_model = cv2.resize(img_np, resize_to, interpolation=cv2.INTER_LINEAR)
        else:
            img_disp = (img_np * 255).astype(np.uint8)
            img_for_model = img_np

        # Apply same PerImageZScore transform you used
        # Using the simple transform you already wrote:
        transformed = PerImageZScore(p=1.0).apply(img_for_model)
        x = torch.from_numpy(transformed).unsqueeze(0).unsqueeze(0).float().to(device)  # (1,1,H,W)

        # --- Run model forward to get segmentation output (prob map) ---
        with torch.set_grad_enabled(True):   # Grad-CAM needs gradients; keep grads enabled
            out = model(x)   # raw logits (1,1,H',W') or (1,1,H,W) depending on model
            # If model output size != input size, interpolate to input size
            out_up = F.interpolate(out, size=img_for_model.shape, mode='bilinear', align_corners=False)
            prob_map = torch.sigmoid(out_up).squeeze().cpu().detach().numpy()  # H x W, float [0,1]

            # --- Grad-CAM ---
            cam_map, _ = gradcam(x, mode='sum')   # cam_map already resized inside gradcam to x.shape[2:]
            # cam_map is numpy HxW (float). Normalize 0..1
            cam_map = cam_map - cam_map.min()
            if cam_map.max() > 0:
                cam_map = cam_map / cam_map.max()
            cam_map = cv2.resize(cam_map, (prob_map.shape[1], prob_map.shape[0]), interpolation=cv2.INTER_LINEAR)

        # --- prepare saves ---
        # # 1) save probability map as 8-bit grayscale
        # prob_uint8 = (np.clip(prob_map, 0.0, 1.0) * 255.0).astype(np.uint8)
        # prob_save_path = os.path.join(mask_out_dir, f'{name_stem}_prob.png')
        # cv2.imwrite(prob_save_path, prob_uint8)

        # 2) save binary mask
        bin_mask = (prob_map >= threshold).astype(np.uint8) * 255
        bin_save_path = os.path.join(mask_out_dir, f'{name_stem}_bin.png')
        cv2.imwrite(bin_save_path, bin_mask)

        # 3) save gradcam (apply color map)
        cam_uint8 = (cam_map * 255.0).astype(np.uint8)
        cam_color = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)  # BGR
        # Option A: save heatmap only
        cam_save_path = os.path.join(grad_out_dir, f'{name_stem}_grad.png')
        cv2.imwrite(cam_save_path, cam_color)

        # # Option B: overlay on grayscale image for better visualization
        # overlay = cv2.cvtColor(img_disp, cv2.COLOR_GRAY2BGR)
        # overlay = cv2.resize(overlay, (cam_color.shape[1], cam_color.shape[0]), interpolation=cv2.INTER_LINEAR)
        # overlay_vis = cv2.addWeighted(overlay, 0.5, cam_color, 0.5, 0)
        # overlay_save_path = os.path.join(grad_out_dir, f'{name_stem}_overlay.png')
        # cv2.imwrite(overlay_save_path, overlay_vis)

        print(f'Processed {name} -> bin:{Path(bin_save_path).name}, grad:{Path(cam_save_path).name}')

# -----------------------
# Example usage (customize paths)
# -----------------------
input_dir = './archive/brisc2025/classification_input/glioma'
mask_out_dir = './archive/brisc2025/classification_input/glioma_masks'
grad_out_dir = './archive/brisc2025/classification_input/glioma_gradcam'

# If your model expects fixed size (e.g., 224x224) set resize_to=(224,224), else None
save_mask_and_gradcam_from_dir(input_dir, mask_out_dir, grad_out_dir, model, grad_cam, resize_to=None, threshold=0.5)

Processed brisc2025_train_00001_gl_ax_t1.jpg -> bin:brisc2025_train_00001_gl_ax_t1_bin.png, grad:brisc2025_train_00001_gl_ax_t1_grad.png
Processed brisc2025_train_00002_gl_ax_t1.jpg -> bin:brisc2025_train_00002_gl_ax_t1_bin.png, grad:brisc2025_train_00002_gl_ax_t1_grad.png
Processed brisc2025_train_00003_gl_ax_t1.jpg -> bin:brisc2025_train_00003_gl_ax_t1_bin.png, grad:brisc2025_train_00003_gl_ax_t1_grad.png
Processed brisc2025_train_00004_gl_ax_t1.jpg -> bin:brisc2025_train_00004_gl_ax_t1_bin.png, grad:brisc2025_train_00004_gl_ax_t1_grad.png
Processed brisc2025_train_00005_gl_ax_t1.jpg -> bin:brisc2025_train_00005_gl_ax_t1_bin.png, grad:brisc2025_train_00005_gl_ax_t1_grad.png
Processed brisc2025_train_00006_gl_ax_t1.jpg -> bin:brisc2025_train_00006_gl_ax_t1_bin.png, grad:brisc2025_train_00006_gl_ax_t1_grad.png
Processed brisc2025_train_00007_gl_ax_t1.jpg -> bin:brisc2025_train_00007_gl_ax_t1_bin.png, grad:brisc2025_train_00007_gl_ax_t1_grad.png
Processed brisc2025_train_00008_gl_ax_t1.

In [10]:

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def save_mask_and_gradcam_from_dir(input_dir,
                                   mask_out_dir,
                                   grad_out_dir,
                                   model,
                                   gradcam,
                                   img_exts=('.jpg', '.jpeg', '.png'),
                                   threshold=0.5,
                                   resize_to=None):
    """
    Processes every image file in input_dir (matching img_exts).
    - saves segmentation probability map and binary mask to mask_out_dir
    - saves Grad-CAM heatmap (colored) to grad_out_dir
    - resize_to: (W,H) or None to keep original size
    """
    ensure_dir(mask_out_dir)
    ensure_dir(grad_out_dir)

    image_paths = []
    for ext in img_exts:
        image_paths += glob(os.path.join(input_dir, f'*{ext}'))
    image_paths = sorted(image_paths)
    if len(image_paths) == 0:
        print('No images found in', input_dir)
        return


    device = next(model.parameters()).device

    for p in image_paths:
        name = Path(p).name
        name_stem = Path(p).stem

        # --- load and preprocess similarly to your test code ---
        pil = Image.open(p).convert('L')
        img_np = np.array(pil).astype(np.float32) / 255.0  # H x W

        # optional resize here to the model's expected input if needed:
        if resize_to is not None:
            img_disp = cv2.resize((img_np * 255).astype(np.uint8), resize_to, interpolation=cv2.INTER_LINEAR)
            img_for_model = cv2.resize(img_np, resize_to, interpolation=cv2.INTER_LINEAR)
        else:
            img_disp = (img_np * 255).astype(np.uint8)
            img_for_model = img_np

        # Apply same PerImageZScore transform you used
        # Using the simple transform you already wrote:
        transformed = PerImageZScore(p=1.0).apply(img_for_model)
        x = torch.from_numpy(transformed).unsqueeze(0).unsqueeze(0).float().to(device)  # (1,1,H,W)

        # --- Run model forward to get segmentation output (prob map) ---
        with torch.set_grad_enabled(True):   # Grad-CAM needs gradients; keep grads enabled
            out = model(x)   # raw logits (1,1,H',W') or (1,1,H,W) depending on model
            # If model output size != input size, interpolate to input size
            out_up = F.interpolate(out, size=img_for_model.shape, mode='bilinear', align_corners=False)
            prob_map = torch.sigmoid(out_up).squeeze().cpu().detach().numpy()  # H x W, float [0,1]

            # --- Grad-CAM ---
            cam_map, _ = gradcam(x, mode='sum')   # cam_map already resized inside gradcam to x.shape[2:]
            # cam_map is numpy HxW (float). Normalize 0..1
            cam_map = cam_map - cam_map.min()
            if cam_map.max() > 0:
                cam_map = cam_map / cam_map.max()
            cam_map = cv2.resize(cam_map, (prob_map.shape[1], prob_map.shape[0]), interpolation=cv2.INTER_LINEAR)

        # # --- prepare saves ---
        # # 1) save probability map as 8-bit grayscale
        # prob_uint8 = (np.clip(prob_map, 0.0, 1.0) * 255.0).astype(np.uint8)
        # prob_save_path = os.path.join(mask_out_dir, f'{name_stem}_prob.png')
        # cv2.imwrite(prob_save_path, prob_uint8)

        # 2) save binary mask
        bin_mask = (prob_map >= threshold).astype(np.uint8) * 255
        bin_save_path = os.path.join(mask_out_dir, f'{name_stem}_bin.png')
        cv2.imwrite(bin_save_path, bin_mask)

        # 3) save gradcam (apply color map)
        cam_uint8 = (cam_map * 255.0).astype(np.uint8)
        cam_color = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)  # BGR
        # Option A: save heatmap only
        cam_save_path = os.path.join(grad_out_dir, f'{name_stem}_grad.png')
        cv2.imwrite(cam_save_path, cam_color)

        # # Option B: overlay on grayscale image for better visualization
        # overlay = cv2.cvtColor(img_disp, cv2.COLOR_GRAY2BGR)
        # overlay = cv2.resize(overlay, (cam_color.shape[1], cam_color.shape[0]), interpolation=cv2.INTER_LINEAR)
        # overlay_vis = cv2.addWeighted(overlay, 0.5, cam_color, 0.5, 0)
        # overlay_save_path = os.path.join(grad_out_dir, f'{name_stem}_overlay.png')
        # cv2.imwrite(overlay_save_path, overlay_vis)

        print(f'Processed {name} -> bin:{Path(bin_save_path).name}, grad:{Path(cam_save_path).name}')

# -----------------------
# Example usage (customize paths)
# -----------------------
input_dir = './archive/brisc2025/classification_input/meningioma'
mask_out_dir = './archive/brisc2025/classification_input/meningioma_masks'
grad_out_dir = './archive/brisc2025/classification_input/meningioma_gradcam'

# If your model expects fixed size (e.g., 224x224) set resize_to=(224,224), else None
save_mask_and_gradcam_from_dir(input_dir, mask_out_dir, grad_out_dir, model, grad_cam, resize_to=None, threshold=0.5)

Processed brisc2025_train_01148_me_ax_t1.jpg -> bin:brisc2025_train_01148_me_ax_t1_bin.png, grad:brisc2025_train_01148_me_ax_t1_grad.png
Processed brisc2025_train_01149_me_ax_t1.jpg -> bin:brisc2025_train_01149_me_ax_t1_bin.png, grad:brisc2025_train_01149_me_ax_t1_grad.png
Processed brisc2025_train_01150_me_ax_t1.jpg -> bin:brisc2025_train_01150_me_ax_t1_bin.png, grad:brisc2025_train_01150_me_ax_t1_grad.png
Processed brisc2025_train_01151_me_ax_t1.jpg -> bin:brisc2025_train_01151_me_ax_t1_bin.png, grad:brisc2025_train_01151_me_ax_t1_grad.png
Processed brisc2025_train_01152_me_ax_t1.jpg -> bin:brisc2025_train_01152_me_ax_t1_bin.png, grad:brisc2025_train_01152_me_ax_t1_grad.png
Processed brisc2025_train_01153_me_ax_t1.jpg -> bin:brisc2025_train_01153_me_ax_t1_bin.png, grad:brisc2025_train_01153_me_ax_t1_grad.png
Processed brisc2025_train_01154_me_ax_t1.jpg -> bin:brisc2025_train_01154_me_ax_t1_bin.png, grad:brisc2025_train_01154_me_ax_t1_grad.png
Processed brisc2025_train_01155_me_ax_t1.

In [11]:

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def save_mask_and_gradcam_from_dir(input_dir,
                                   mask_out_dir,
                                   grad_out_dir,
                                   model,
                                   gradcam,
                                   img_exts=('.jpg', '.jpeg', '.png'),
                                   threshold=0.5,
                                   resize_to=None):
    """
    Processes every image file in input_dir (matching img_exts).
    - saves segmentation probability map and binary mask to mask_out_dir
    - saves Grad-CAM heatmap (colored) to grad_out_dir
    - resize_to: (W,H) or None to keep original size
    """
    ensure_dir(mask_out_dir)
    ensure_dir(grad_out_dir)

    image_paths = []
    for ext in img_exts:
        image_paths += glob(os.path.join(input_dir, f'*{ext}'))
    image_paths = sorted(image_paths)
    if len(image_paths) == 0:
        print('No images found in', input_dir)
        return


    device = next(model.parameters()).device

    for p in image_paths:
        name = Path(p).name
        name_stem = Path(p).stem

        # --- load and preprocess similarly to your test code ---
        pil = Image.open(p).convert('L')
        img_np = np.array(pil).astype(np.float32) / 255.0  # H x W

        # optional resize here to the model's expected input if needed:
        if resize_to is not None:
            img_disp = cv2.resize((img_np * 255).astype(np.uint8), resize_to, interpolation=cv2.INTER_LINEAR)
            img_for_model = cv2.resize(img_np, resize_to, interpolation=cv2.INTER_LINEAR)
        else:
            img_disp = (img_np * 255).astype(np.uint8)
            img_for_model = img_np

        # Apply same PerImageZScore transform you used
        # Using the simple transform you already wrote:
        transformed = PerImageZScore(p=1.0).apply(img_for_model)
        x = torch.from_numpy(transformed).unsqueeze(0).unsqueeze(0).float().to(device)  # (1,1,H,W)

        # --- Run model forward to get segmentation output (prob map) ---
        with torch.set_grad_enabled(True):   # Grad-CAM needs gradients; keep grads enabled
            out = model(x)   # raw logits (1,1,H',W') or (1,1,H,W) depending on model
            # If model output size != input size, interpolate to input size
            out_up = F.interpolate(out, size=img_for_model.shape, mode='bilinear', align_corners=False)
            prob_map = torch.sigmoid(out_up).squeeze().cpu().detach().numpy()  # H x W, float [0,1]

            # --- Grad-CAM ---
            cam_map, _ = gradcam(x, mode='sum')   # cam_map already resized inside gradcam to x.shape[2:]
            # cam_map is numpy HxW (float). Normalize 0..1
            cam_map = cam_map - cam_map.min()
            if cam_map.max() > 0:
                cam_map = cam_map / cam_map.max()
            cam_map = cv2.resize(cam_map, (prob_map.shape[1], prob_map.shape[0]), interpolation=cv2.INTER_LINEAR)

        # --- prepare saves ---
        # # 1) save probability map as 8-bit grayscale
        # prob_uint8 = (np.clip(prob_map, 0.0, 1.0) * 255.0).astype(np.uint8)
        # prob_save_path = os.path.join(mask_out_dir, f'{name_stem}_prob.png')
        # cv2.imwrite(prob_save_path, prob_uint8)

        # 2) save binary mask
        bin_mask = (prob_map >= threshold).astype(np.uint8) * 255
        bin_save_path = os.path.join(mask_out_dir, f'{name_stem}_bin.png')
        cv2.imwrite(bin_save_path, bin_mask)

        # 3) save gradcam (apply color map)
        cam_uint8 = (cam_map * 255.0).astype(np.uint8)
        cam_color = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)  # BGR
        # Option A: save heatmap only
        cam_save_path = os.path.join(grad_out_dir, f'{name_stem}_grad.png')
        cv2.imwrite(cam_save_path, cam_color)

        # # Option B: overlay on grayscale image for better visualization
        # overlay = cv2.cvtColor(img_disp, cv2.COLOR_GRAY2BGR)
        # overlay = cv2.resize(overlay, (cam_color.shape[1], cam_color.shape[0]), interpolation=cv2.INTER_LINEAR)
        # overlay_vis = cv2.addWeighted(overlay, 0.5, cam_color, 0.5, 0)
        # overlay_save_path = os.path.join(grad_out_dir, f'{name_stem}_overlay.png')
        # cv2.imwrite(overlay_save_path, overlay_vis)

        print(f'Processed {name} -> bin:{Path(bin_save_path).name}, grad:{Path(cam_save_path).name}')

# -----------------------
# Example usage (customize paths)
# -----------------------
input_dir = './archive/brisc2025/classification_input/pituitary'
mask_out_dir = './archive/brisc2025/classification_input/pituitary_masks'
grad_out_dir = './archive/brisc2025/classification_input/pituitary_gradcam'

# If your model expects fixed size (e.g., 224x224) set resize_to=(224,224), else None
save_mask_and_gradcam_from_dir(input_dir, mask_out_dir, grad_out_dir, model, grad_cam, resize_to=None, threshold=0.5)

Processed brisc2025_train_03544_pi_ax_t1.jpg -> bin:brisc2025_train_03544_pi_ax_t1_bin.png, grad:brisc2025_train_03544_pi_ax_t1_grad.png
Processed brisc2025_train_03545_pi_ax_t1.jpg -> bin:brisc2025_train_03545_pi_ax_t1_bin.png, grad:brisc2025_train_03545_pi_ax_t1_grad.png
Processed brisc2025_train_03546_pi_ax_t1.jpg -> bin:brisc2025_train_03546_pi_ax_t1_bin.png, grad:brisc2025_train_03546_pi_ax_t1_grad.png
Processed brisc2025_train_03547_pi_ax_t1.jpg -> bin:brisc2025_train_03547_pi_ax_t1_bin.png, grad:brisc2025_train_03547_pi_ax_t1_grad.png
Processed brisc2025_train_03548_pi_ax_t1.jpg -> bin:brisc2025_train_03548_pi_ax_t1_bin.png, grad:brisc2025_train_03548_pi_ax_t1_grad.png
Processed brisc2025_train_03549_pi_ax_t1.jpg -> bin:brisc2025_train_03549_pi_ax_t1_bin.png, grad:brisc2025_train_03549_pi_ax_t1_grad.png
Processed brisc2025_train_03550_pi_ax_t1.jpg -> bin:brisc2025_train_03550_pi_ax_t1_bin.png, grad:brisc2025_train_03550_pi_ax_t1_grad.png
Processed brisc2025_train_03551_pi_ax_t1.

In [12]:

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def save_mask_and_gradcam_from_dir(input_dir,
                                   mask_out_dir,
                                   grad_out_dir,
                                   model,
                                   gradcam,
                                   img_exts=('.jpg', '.jpeg', '.png'),
                                   threshold=0.5,
                                   resize_to=None):
    """
    Processes every image file in input_dir (matching img_exts).
    - saves segmentation probability map and binary mask to mask_out_dir
    - saves Grad-CAM heatmap (colored) to grad_out_dir
    - resize_to: (W,H) or None to keep original size
    """
    ensure_dir(mask_out_dir)
    ensure_dir(grad_out_dir)

    image_paths = []
    for ext in img_exts:
        image_paths += glob(os.path.join(input_dir, f'*{ext}'))
    image_paths = sorted(image_paths)
    if len(image_paths) == 0:
        print('No images found in', input_dir)
        return


    device = next(model.parameters()).device

    for p in image_paths:
        name = Path(p).name
        name_stem = Path(p).stem

        # --- load and preprocess similarly to your test code ---
        pil = Image.open(p).convert('L')
        img_np = np.array(pil).astype(np.float32) / 255.0  # H x W

        # optional resize here to the model's expected input if needed:
        if resize_to is not None:
            img_disp = cv2.resize((img_np * 255).astype(np.uint8), resize_to, interpolation=cv2.INTER_LINEAR)
            img_for_model = cv2.resize(img_np, resize_to, interpolation=cv2.INTER_LINEAR)
        else:
            img_disp = (img_np * 255).astype(np.uint8)
            img_for_model = img_np

        # Apply same PerImageZScore transform you used
        # Using the simple transform you already wrote:
        transformed = PerImageZScore(p=1.0).apply(img_for_model)
        x = torch.from_numpy(transformed).unsqueeze(0).unsqueeze(0).float().to(device)  # (1,1,H,W)

        # --- Run model forward to get segmentation output (prob map) ---
        with torch.set_grad_enabled(True):   # Grad-CAM needs gradients; keep grads enabled
            out = model(x)   # raw logits (1,1,H',W') or (1,1,H,W) depending on model
            # If model output size != input size, interpolate to input size
            out_up = F.interpolate(out, size=img_for_model.shape, mode='bilinear', align_corners=False)
            prob_map = torch.sigmoid(out_up).squeeze().cpu().detach().numpy()  # H x W, float [0,1]

            # --- Grad-CAM ---
            cam_map, _ = gradcam(x, mode='sum')   # cam_map already resized inside gradcam to x.shape[2:]
            # cam_map is numpy HxW (float). Normalize 0..1
            cam_map = cam_map - cam_map.min()
            if cam_map.max() > 0:
                cam_map = cam_map / cam_map.max()
            cam_map = cv2.resize(cam_map, (prob_map.shape[1], prob_map.shape[0]), interpolation=cv2.INTER_LINEAR)

        # --- prepare saves ---
        # # 1) save probability map as 8-bit grayscale
        # prob_uint8 = (np.clip(prob_map, 0.0, 1.0) * 255.0).astype(np.uint8)
        # prob_save_path = os.path.join(mask_out_dir, f'{name_stem}_prob.png')
        # cv2.imwrite(prob_save_path, prob_uint8)

        # 2) save binary mask
        bin_mask = (prob_map >= threshold).astype(np.uint8) * 255
        bin_save_path = os.path.join(mask_out_dir, f'{name_stem}_bin.png')
        cv2.imwrite(bin_save_path, bin_mask)

        # 3) save gradcam (apply color map)
        cam_uint8 = (cam_map * 255.0).astype(np.uint8)
        cam_color = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)  # BGR
        # Option A: save heatmap only
        cam_save_path = os.path.join(grad_out_dir, f'{name_stem}_grad.png')
        cv2.imwrite(cam_save_path, cam_color)

        # # Option B: overlay on grayscale image for better visualization
        # overlay = cv2.cvtColor(img_disp, cv2.COLOR_GRAY2BGR)
        # overlay = cv2.resize(overlay, (cam_color.shape[1], cam_color.shape[0]), interpolation=cv2.INTER_LINEAR)
        # overlay_vis = cv2.addWeighted(overlay, 0.5, cam_color, 0.5, 0)
        # overlay_save_path = os.path.join(grad_out_dir, f'{name_stem}_overlay.png')
        # cv2.imwrite(overlay_save_path, overlay_vis)

        print(f'Processed {name} -> bin:{Path(bin_save_path).name}, grad:{Path(cam_save_path).name}')

# -----------------------
# Example usage (customize paths)
# -----------------------
input_dir = './archive/brisc2025/classification_input/no_tumor'
mask_out_dir = './archive/brisc2025/classification_input/no_tumor_masks'
grad_out_dir = './archive/brisc2025/classification_input/no_tumor_gradcam'

# If your model expects fixed size (e.g., 224x224) set resize_to=(224,224), else None
save_mask_and_gradcam_from_dir(input_dir, mask_out_dir, grad_out_dir, model, grad_cam, resize_to=None, threshold=0.5)

Processed brisc2025_train_02477_no_ax_t1.jpg -> bin:brisc2025_train_02477_no_ax_t1_bin.png, grad:brisc2025_train_02477_no_ax_t1_grad.png
Processed brisc2025_train_02478_no_ax_t1.jpg -> bin:brisc2025_train_02478_no_ax_t1_bin.png, grad:brisc2025_train_02478_no_ax_t1_grad.png
Processed brisc2025_train_02479_no_ax_t1.jpg -> bin:brisc2025_train_02479_no_ax_t1_bin.png, grad:brisc2025_train_02479_no_ax_t1_grad.png
Processed brisc2025_train_02480_no_ax_t1.jpg -> bin:brisc2025_train_02480_no_ax_t1_bin.png, grad:brisc2025_train_02480_no_ax_t1_grad.png
Processed brisc2025_train_02481_no_ax_t1.jpg -> bin:brisc2025_train_02481_no_ax_t1_bin.png, grad:brisc2025_train_02481_no_ax_t1_grad.png
Processed brisc2025_train_02482_no_ax_t1.jpg -> bin:brisc2025_train_02482_no_ax_t1_bin.png, grad:brisc2025_train_02482_no_ax_t1_grad.png
Processed brisc2025_train_02483_no_ax_t1.jpg -> bin:brisc2025_train_02483_no_ax_t1_bin.png, grad:brisc2025_train_02483_no_ax_t1_grad.png
Processed brisc2025_train_02484_no_ax_t1.

In [19]:
!python build_labels_csv_3ch.py --root /content/archive/brisc2025/classification_input --out /content/archive/brisc2025/labels.csv --val-fraction 0.12

[WARN] no images for class .ipynb_checkpoints at /content/archive/brisc2025/classification_input/.ipynb_checkpoints/images, skipping
[WARN] no images for class placeholders_for_missing at /content/archive/brisc2025/classification_input/placeholders_for_missing/images, skipping
Wrote 5000 rows to /content/archive/brisc2025/labels.csv
Label map: {'glioma': 0, 'meningioma': 1, 'no_tumor': 2, 'pituitary': 3}


In [20]:
import pandas as pd
df = pd.read_csv('/content/archive/brisc2025/labels.csv')
df.head(8)

,image_path,mask_path,gradcam_path,label,class_name,split
0,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,0,glioma,train
1,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,0,glioma,train
2,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,0,glioma,train
3,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,0,glioma,train
4,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,0,glioma,train
5,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,0,glioma,train
6,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,0,glioma,train
7,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,/content/archive/brisc2025/classification_inpu...,0,glioma,train


In [22]:
import pandas as pd, cv2, numpy as np
df = pd.read_csv('/content/archive/brisc2025/labels.csv')
row = df[df['split']=='train'].iloc[0]
print(row['image_path'], row['mask_path'], row['gradcam_path'], row['label'])
img = cv2.imread(row['image_path'], cv2.IMREAD_GRAYSCALE)
mask = cv2.imread(row['mask_path'], cv2.IMREAD_GRAYSCALE)
gcam = cv2.imread(row['gradcam_path'], cv2.IMREAD_GRAYSCALE)
print('shapes before resizing:', img.shape, mask.shape, gcam.shape)

# Resize mask and gcam to match img.shape
if img.shape != mask.shape:
    mask = cv2.resize(mask, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_NEAREST) # For masks, use NEAREST to preserve discrete values
if img.shape != gcam.shape:
    gcam = cv2.resize(gcam, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR) # For gradcam, use LINEAR for smooth interpolation

print('shapes after resizing:', img.shape, mask.shape, gcam.shape)
stack = np.stack([(img.astype('float32')/255.0), (mask.astype('float32')/255.0), (gcam.astype('float32')/255.0)], axis=0)  # CHW
print('stack CHW:', stack.shape)

/content/archive/brisc2025/classification_input/glioma/images/brisc2025_train_00629_gl_co_t1.jpg /content/archive/brisc2025/classification_input/placeholders_for_missing/glioma_missing_mask.png /content/archive/brisc2025/classification_input/placeholders_for_missing/glioma_missing_gradcam.png 0
shapes before resizing: (256, 256) (224, 224) (224, 224)
shapes after resizing: (256, 256) (256, 256) (256, 256)
stack CHW: (3, 256, 256)


In [32]:
!python /content/resnet18_colab_train_3ch.py \
  --data-csv /content/archive/brisc2025/labels.csv \
  --batch-size 16 \
  --epochs 20 \
  --img-size 256 \
  --checkpoint-dir /content/archive/brisc2025/checkpoints \
  --use-amp \
  --weighted-sampler

Overwriting /content/resnet18_colab_train_3ch.py


In [35]:
!python /content/resnet18_colab_train_3ch.py \
  --data-csv /content/archive/brisc2025/labels.csv \
  --batch-size 32 \
  --epochs 20 \
  --img-size 256 \
  --checkpoint-dir /content/archive/brisc2025/checkpoints \
  --use-amp \
  --weighted-sampler \
  --num-workers 0

Train samples: 4402  Val samples: 598
Classes: ['glioma', 'meningioma', 'no_tumor', 'pituitary']
/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
Using WeightedRandomSampler for train loader
Device: cuda
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(

In [36]:
# Set paths

image_input_dir = "/content/archive/brisc2025/classification_task/test/glioma"          # folder containing original .jpg images
mask_input_dir = "/content/archive/brisc2025/classification_task/test/pituitary1"            # folder containing original .png masks

image_output_dir = "/content/archive/brisc2025/classification_test/glioma" # folder to save resized images
mask_output_dir = "/content/archive/brisc2025/classification_test/pituitary1"   # folder to save resized masks

# Desired output size
new_size = (256, 256)   # (width, height)

# Create output directories if they don't exist
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(mask_output_dir, exist_ok=True)


def resize_image(input_path, output_path, size, is_mask=False):
    img = Image.open(input_path)

    # For masks → nearest neighbor (keeps integer class values)
    if is_mask:
        img = img.resize(size, Image.NEAREST)
    else:
        # For images → bilinear interpolation
        img = img.resize(size, Image.BILINEAR)

    img.save(output_path)


# Resize all JPG images
for filename in os.listdir(image_input_dir):
    if filename.lower().endswith(".jpg"):
        input_path = os.path.join(image_input_dir, filename)
        output_path = os.path.join(image_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=False)
        print("Resized image:", filename)

# Resize all PNG masks
for filename in os.listdir(mask_input_dir):
    if filename.lower().endswith(".png"):
        input_path = os.path.join(mask_input_dir, filename)
        output_path = os.path.join(mask_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=True)
        print("Resized mask:", filename)

print("Done! Images and masks resized and saved.")

Resized image: brisc2025_test_00129_gl_co_t1.jpg
Resized image: brisc2025_test_00151_gl_co_t1.jpg
Resized image: brisc2025_test_00172_gl_sa_t1.jpg
Resized image: brisc2025_test_00033_gl_ax_t1.jpg
Resized image: brisc2025_test_00021_gl_ax_t1.jpg
Resized image: brisc2025_test_00026_gl_ax_t1.jpg
Resized image: brisc2025_test_00091_gl_co_t1.jpg
Resized image: brisc2025_test_00147_gl_co_t1.jpg
Resized image: brisc2025_test_00109_gl_co_t1.jpg
Resized image: brisc2025_test_00132_gl_co_t1.jpg
Resized image: brisc2025_test_00170_gl_sa_t1.jpg
Resized image: brisc2025_test_00044_gl_ax_t1.jpg
Resized image: brisc2025_test_00031_gl_ax_t1.jpg
Resized image: brisc2025_test_00179_gl_sa_t1.jpg
Resized image: brisc2025_test_00010_gl_ax_t1.jpg
Resized image: brisc2025_test_00174_gl_sa_t1.jpg
Resized image: brisc2025_test_00013_gl_ax_t1.jpg
Resized image: brisc2025_test_00117_gl_co_t1.jpg
Resized image: brisc2025_test_00247_gl_sa_t1.jpg
Resized image: brisc2025_test_00198_gl_sa_t1.jpg
Resized image: brisc

FileNotFoundError: [Errno 2] No such file or directory: '/content/archive/brisc2025/classification_task/test/pituitary1'

In [37]:
# Set paths

image_input_dir = "/content/archive/brisc2025/classification_task/test/meningioma"          # folder containing original .jpg images
mask_input_dir = "/content/archive/brisc2025/classification_task/test/pituitary1"            # folder containing original .png masks

image_output_dir = "/content/archive/brisc2025/classification_test/meningioma/images" # folder to save resized images
mask_output_dir = "/content/archive/brisc2025/classification_test/pituitary1"   # folder to save resized masks

# Desired output size
new_size = (256, 256)   # (width, height)

# Create output directories if they don't exist
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(mask_output_dir, exist_ok=True)


def resize_image(input_path, output_path, size, is_mask=False):
    img = Image.open(input_path)

    # For masks → nearest neighbor (keeps integer class values)
    if is_mask:
        img = img.resize(size, Image.NEAREST)
    else:
        # For images → bilinear interpolation
        img = img.resize(size, Image.BILINEAR)

    img.save(output_path)


# Resize all JPG images
for filename in os.listdir(image_input_dir):
    if filename.lower().endswith(".jpg"):
        input_path = os.path.join(image_input_dir, filename)
        output_path = os.path.join(image_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=False)
        print("Resized image:", filename)

# Resize all PNG masks
for filename in os.listdir(mask_input_dir):
    if filename.lower().endswith(".png"):
        input_path = os.path.join(mask_input_dir, filename)
        output_path = os.path.join(mask_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=True)
        print("Resized mask:", filename)

print("Done! Images and masks resized and saved.")

Resized image: brisc2025_test_00385_me_ax_t1.jpg
Resized image: brisc2025_test_00486_me_sa_t1.jpg
Resized image: brisc2025_test_00477_me_co_t1.jpg
Resized image: brisc2025_test_00390_me_ax_t1.jpg
Resized image: brisc2025_test_00515_me_sa_t1.jpg
Resized image: brisc2025_test_00423_me_co_t1.jpg
Resized image: brisc2025_test_00495_me_sa_t1.jpg
Resized image: brisc2025_test_00458_me_co_t1.jpg
Resized image: brisc2025_test_00504_me_sa_t1.jpg
Resized image: brisc2025_test_00402_me_co_t1.jpg
Resized image: brisc2025_test_00262_me_ax_t1.jpg
Resized image: brisc2025_test_00297_me_ax_t1.jpg
Resized image: brisc2025_test_00326_me_ax_t1.jpg
Resized image: brisc2025_test_00480_me_sa_t1.jpg
Resized image: brisc2025_test_00444_me_co_t1.jpg
Resized image: brisc2025_test_00422_me_co_t1.jpg
Resized image: brisc2025_test_00449_me_co_t1.jpg
Resized image: brisc2025_test_00336_me_ax_t1.jpg
Resized image: brisc2025_test_00286_me_ax_t1.jpg
Resized image: brisc2025_test_00414_me_co_t1.jpg
Resized image: brisc

FileNotFoundError: [Errno 2] No such file or directory: '/content/archive/brisc2025/classification_task/test/pituitary1'

In [38]:
# Set paths

image_input_dir = "/content/archive/brisc2025/classification_task/test/no_tumor"          # folder containing original .jpg images
mask_input_dir = "/content/archive/brisc2025/classification_task/test/pituitary1"            # folder containing original .png masks

image_output_dir = "/content/archive/brisc2025/classification_test/no_tumor/images" # folder to save resized images
mask_output_dir = "/content/archive/brisc2025/classification_test/pituitary1"   # folder to save resized masks

# Desired output size
new_size = (256, 256)   # (width, height)

# Create output directories if they don't exist
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(mask_output_dir, exist_ok=True)


def resize_image(input_path, output_path, size, is_mask=False):
    img = Image.open(input_path)

    # For masks → nearest neighbor (keeps integer class values)
    if is_mask:
        img = img.resize(size, Image.NEAREST)
    else:
        # For images → bilinear interpolation
        img = img.resize(size, Image.BILINEAR)

    img.save(output_path)


# Resize all JPG images
for filename in os.listdir(image_input_dir):
    if filename.lower().endswith(".jpg"):
        input_path = os.path.join(image_input_dir, filename)
        output_path = os.path.join(image_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=False)
        print("Resized image:", filename)

# Resize all PNG masks
for filename in os.listdir(mask_input_dir):
    if filename.lower().endswith(".png"):
        input_path = os.path.join(mask_input_dir, filename)
        output_path = os.path.join(mask_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=True)
        print("Resized mask:", filename)

print("Done! Images and masks resized and saved.")

Resized image: brisc2025_test_00624_no_co_t1.jpg
Resized image: brisc2025_test_00592_no_ax_t1.jpg
Resized image: brisc2025_test_00583_no_ax_t1.jpg
Resized image: brisc2025_test_00629_no_co_t1.jpg
Resized image: brisc2025_test_00690_no_sa_t1.jpg
Resized image: brisc2025_test_00562_no_ax_t1.jpg
Resized image: brisc2025_test_00694_no_sa_t1.jpg
Resized image: brisc2025_test_00565_no_ax_t1.jpg
Resized image: brisc2025_test_00603_no_ax_t1.jpg
Resized image: brisc2025_test_00581_no_ax_t1.jpg
Resized image: brisc2025_test_00683_no_sa_t1.jpg
Resized image: brisc2025_test_00612_no_ax_t1.jpg
Resized image: brisc2025_test_00659_no_co_t1.jpg
Resized image: brisc2025_test_00661_no_sa_t1.jpg
Resized image: brisc2025_test_00697_no_sa_t1.jpg
Resized image: brisc2025_test_00596_no_ax_t1.jpg
Resized image: brisc2025_test_00669_no_sa_t1.jpg
Resized image: brisc2025_test_00676_no_sa_t1.jpg
Resized image: brisc2025_test_00646_no_co_t1.jpg
Resized image: brisc2025_test_00606_no_ax_t1.jpg
Resized image: brisc

FileNotFoundError: [Errno 2] No such file or directory: '/content/archive/brisc2025/classification_task/test/pituitary1'

In [39]:
# Set paths

image_input_dir = "/content/archive/brisc2025/classification_task/test/pituitary"          # folder containing original .jpg images
mask_input_dir = "/content/archive/brisc2025/classification_task/test/pituitary1"            # folder containing original .png masks

image_output_dir = "/content/archive/brisc2025/classification_test/pituitary/images" # folder to save resized images
mask_output_dir = "/content/archive/brisc2025/classification_test/pituitary1"   # folder to save resized masks

# Desired output size
new_size = (256, 256)   # (width, height)

# Create output directories if they don't exist
os.makedirs(image_output_dir, exist_ok=True)
os.makedirs(mask_output_dir, exist_ok=True)


def resize_image(input_path, output_path, size, is_mask=False):
    img = Image.open(input_path)

    # For masks → nearest neighbor (keeps integer class values)
    if is_mask:
        img = img.resize(size, Image.NEAREST)
    else:
        # For images → bilinear interpolation
        img = img.resize(size, Image.BILINEAR)

    img.save(output_path)


# Resize all JPG images
for filename in os.listdir(image_input_dir):
    if filename.lower().endswith(".jpg"):
        input_path = os.path.join(image_input_dir, filename)
        output_path = os.path.join(image_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=False)
        print("Resized image:", filename)

# Resize all PNG masks
for filename in os.listdir(mask_input_dir):
    if filename.lower().endswith(".png"):
        input_path = os.path.join(mask_input_dir, filename)
        output_path = os.path.join(mask_output_dir, filename)
        resize_image(input_path, output_path, new_size, is_mask=True)
        print("Resized mask:", filename)

print("Done! Images and masks resized and saved.")

Resized image: brisc2025_test_00759_pi_ax_t1.jpg
Resized image: brisc2025_test_00797_pi_ax_t1.jpg
Resized image: brisc2025_test_00764_pi_ax_t1.jpg
Resized image: brisc2025_test_00864_pi_co_t1.jpg
Resized image: brisc2025_test_00897_pi_co_t1.jpg
Resized image: brisc2025_test_00724_pi_ax_t1.jpg
Resized image: brisc2025_test_00965_pi_sa_t1.jpg
Resized image: brisc2025_test_00969_pi_sa_t1.jpg
Resized image: brisc2025_test_00752_pi_ax_t1.jpg
Resized image: brisc2025_test_00712_pi_ax_t1.jpg
Resized image: brisc2025_test_00892_pi_co_t1.jpg
Resized image: brisc2025_test_00717_pi_ax_t1.jpg
Resized image: brisc2025_test_00758_pi_ax_t1.jpg
Resized image: brisc2025_test_00882_pi_co_t1.jpg
Resized image: brisc2025_test_00922_pi_sa_t1.jpg
Resized image: brisc2025_test_00891_pi_co_t1.jpg
Resized image: brisc2025_test_00779_pi_ax_t1.jpg
Resized image: brisc2025_test_00808_pi_ax_t1.jpg
Resized image: brisc2025_test_00828_pi_co_t1.jpg
Resized image: brisc2025_test_00979_pi_sa_t1.jpg
Resized image: brisc

FileNotFoundError: [Errno 2] No such file or directory: '/content/archive/brisc2025/classification_task/test/pituitary1'

In [40]:

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def save_mask_and_gradcam_from_dir(input_dir,
                                   mask_out_dir,
                                   grad_out_dir,
                                   model,
                                   gradcam,
                                   img_exts=('.jpg', '.jpeg', '.png'),
                                   threshold=0.5,
                                   resize_to=None):
    """
    Processes every image file in input_dir (matching img_exts).
    - saves segmentation probability map and binary mask to mask_out_dir
    - saves Grad-CAM heatmap (colored) to grad_out_dir
    - resize_to: (W,H) or None to keep original size
    """
    ensure_dir(mask_out_dir)
    ensure_dir(grad_out_dir)

    image_paths = []
    for ext in img_exts:
        image_paths += glob(os.path.join(input_dir, f'*{ext}'))
    image_paths = sorted(image_paths)
    if len(image_paths) == 0:
        print('No images found in', input_dir)
        return


    device = next(model.parameters()).device

    for p in image_paths:
        name = Path(p).name
        name_stem = Path(p).stem

        # --- load and preprocess similarly to your test code ---
        pil = Image.open(p).convert('L')
        img_np = np.array(pil).astype(np.float32) / 255.0  # H x W

        # optional resize here to the model's expected input if needed:
        if resize_to is not None:
            img_disp = cv2.resize((img_np * 255).astype(np.uint8), resize_to, interpolation=cv2.INTER_LINEAR)
            img_for_model = cv2.resize(img_np, resize_to, interpolation=cv2.INTER_LINEAR)
        else:
            img_disp = (img_np * 255).astype(np.uint8)
            img_for_model = img_np

        # Apply same PerImageZScore transform you used
        # Using the simple transform you already wrote:
        transformed = PerImageZScore(p=1.0).apply(img_for_model)
        x = torch.from_numpy(transformed).unsqueeze(0).unsqueeze(0).float().to(device)  # (1,1,H,W)

        # --- Run model forward to get segmentation output (prob map) ---
        with torch.set_grad_enabled(True):   # Grad-CAM needs gradients; keep grads enabled
            out = model(x)   # raw logits (1,1,H',W') or (1,1,H,W) depending on model
            # If model output size != input size, interpolate to input size
            out_up = F.interpolate(out, size=img_for_model.shape, mode='bilinear', align_corners=False)
            prob_map = torch.sigmoid(out_up).squeeze().cpu().detach().numpy()  # H x W, float [0,1]

            # --- Grad-CAM ---
            cam_map, _ = gradcam(x, mode='sum')   # cam_map already resized inside gradcam to x.shape[2:]
            # cam_map is numpy HxW (float). Normalize 0..1
            cam_map = cam_map - cam_map.min()
            if cam_map.max() > 0:
                cam_map = cam_map / cam_map.max()
            cam_map = cv2.resize(cam_map, (prob_map.shape[1], prob_map.shape[0]), interpolation=cv2.INTER_LINEAR)

        # --- prepare saves ---
        # # 1) save probability map as 8-bit grayscale
        # prob_uint8 = (np.clip(prob_map, 0.0, 1.0) * 255.0).astype(np.uint8)
        # prob_save_path = os.path.join(mask_out_dir, f'{name_stem}_prob.png')
        # cv2.imwrite(prob_save_path, prob_uint8)

        # 2) save binary mask
        bin_mask = (prob_map >= threshold).astype(np.uint8) * 255
        bin_save_path = os.path.join(mask_out_dir, f'{name_stem}_bin.png')
        cv2.imwrite(bin_save_path, bin_mask)

        # 3) save gradcam (apply color map)
        cam_uint8 = (cam_map * 255.0).astype(np.uint8)
        cam_color = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)  # BGR
        # Option A: save heatmap only
        cam_save_path = os.path.join(grad_out_dir, f'{name_stem}_grad.png')
        cv2.imwrite(cam_save_path, cam_color)

        # # Option B: overlay on grayscale image for better visualization
        # overlay = cv2.cvtColor(img_disp, cv2.COLOR_GRAY2BGR)
        # overlay = cv2.resize(overlay, (cam_color.shape[1], cam_color.shape[0]), interpolation=cv2.INTER_LINEAR)
        # overlay_vis = cv2.addWeighted(overlay, 0.5, cam_color, 0.5, 0)
        # overlay_save_path = os.path.join(grad_out_dir, f'{name_stem}_overlay.png')
        # cv2.imwrite(overlay_save_path, overlay_vis)

        print(f'Processed {name} -> bin:{Path(bin_save_path).name}, grad:{Path(cam_save_path).name}')

# -----------------------
# Example usage (customize paths)
# -----------------------
input_dir = './archive/brisc2025/classification_test/glioma/images'
mask_out_dir = './archive/brisc2025/classification_test/glioma/masks'
grad_out_dir = './archive/brisc2025/classification_test/glioma/gradcams'

# If your model expects fixed size (e.g., 224x224) set resize_to=(224,224), else None
save_mask_and_gradcam_from_dir(input_dir, mask_out_dir, grad_out_dir, model, grad_cam, resize_to=None, threshold=0.5)

Processed brisc2025_test_00001_gl_ax_t1.jpg -> bin:brisc2025_test_00001_gl_ax_t1_bin.png, grad:brisc2025_test_00001_gl_ax_t1_grad.png
Processed brisc2025_test_00002_gl_ax_t1.jpg -> bin:brisc2025_test_00002_gl_ax_t1_bin.png, grad:brisc2025_test_00002_gl_ax_t1_grad.png
Processed brisc2025_test_00003_gl_ax_t1.jpg -> bin:brisc2025_test_00003_gl_ax_t1_bin.png, grad:brisc2025_test_00003_gl_ax_t1_grad.png
Processed brisc2025_test_00004_gl_ax_t1.jpg -> bin:brisc2025_test_00004_gl_ax_t1_bin.png, grad:brisc2025_test_00004_gl_ax_t1_grad.png
Processed brisc2025_test_00005_gl_ax_t1.jpg -> bin:brisc2025_test_00005_gl_ax_t1_bin.png, grad:brisc2025_test_00005_gl_ax_t1_grad.png
Processed brisc2025_test_00006_gl_ax_t1.jpg -> bin:brisc2025_test_00006_gl_ax_t1_bin.png, grad:brisc2025_test_00006_gl_ax_t1_grad.png
Processed brisc2025_test_00007_gl_ax_t1.jpg -> bin:brisc2025_test_00007_gl_ax_t1_bin.png, grad:brisc2025_test_00007_gl_ax_t1_grad.png
Processed brisc2025_test_00008_gl_ax_t1.jpg -> bin:brisc2025_t

In [41]:

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def save_mask_and_gradcam_from_dir(input_dir,
                                   mask_out_dir,
                                   grad_out_dir,
                                   model,
                                   gradcam,
                                   img_exts=('.jpg', '.jpeg', '.png'),
                                   threshold=0.5,
                                   resize_to=None):
    """
    Processes every image file in input_dir (matching img_exts).
    - saves segmentation probability map and binary mask to mask_out_dir
    - saves Grad-CAM heatmap (colored) to grad_out_dir
    - resize_to: (W,H) or None to keep original size
    """
    ensure_dir(mask_out_dir)
    ensure_dir(grad_out_dir)

    image_paths = []
    for ext in img_exts:
        image_paths += glob(os.path.join(input_dir, f'*{ext}'))
    image_paths = sorted(image_paths)
    if len(image_paths) == 0:
        print('No images found in', input_dir)
        return


    device = next(model.parameters()).device

    for p in image_paths:
        name = Path(p).name
        name_stem = Path(p).stem

        # --- load and preprocess similarly to your test code ---
        pil = Image.open(p).convert('L')
        img_np = np.array(pil).astype(np.float32) / 255.0  # H x W

        # optional resize here to the model's expected input if needed:
        if resize_to is not None:
            img_disp = cv2.resize((img_np * 255).astype(np.uint8), resize_to, interpolation=cv2.INTER_LINEAR)
            img_for_model = cv2.resize(img_np, resize_to, interpolation=cv2.INTER_LINEAR)
        else:
            img_disp = (img_np * 255).astype(np.uint8)
            img_for_model = img_np

        # Apply same PerImageZScore transform you used
        # Using the simple transform you already wrote:
        transformed = PerImageZScore(p=1.0).apply(img_for_model)
        x = torch.from_numpy(transformed).unsqueeze(0).unsqueeze(0).float().to(device)  # (1,1,H,W)

        # --- Run model forward to get segmentation output (prob map) ---
        with torch.set_grad_enabled(True):   # Grad-CAM needs gradients; keep grads enabled
            out = model(x)   # raw logits (1,1,H',W') or (1,1,H,W) depending on model
            # If model output size != input size, interpolate to input size
            out_up = F.interpolate(out, size=img_for_model.shape, mode='bilinear', align_corners=False)
            prob_map = torch.sigmoid(out_up).squeeze().cpu().detach().numpy()  # H x W, float [0,1]

            # --- Grad-CAM ---
            cam_map, _ = gradcam(x, mode='sum')   # cam_map already resized inside gradcam to x.shape[2:]
            # cam_map is numpy HxW (float). Normalize 0..1
            cam_map = cam_map - cam_map.min()
            if cam_map.max() > 0:
                cam_map = cam_map / cam_map.max()
            cam_map = cv2.resize(cam_map, (prob_map.shape[1], prob_map.shape[0]), interpolation=cv2.INTER_LINEAR)

        # --- prepare saves ---
        # # 1) save probability map as 8-bit grayscale
        # prob_uint8 = (np.clip(prob_map, 0.0, 1.0) * 255.0).astype(np.uint8)
        # prob_save_path = os.path.join(mask_out_dir, f'{name_stem}_prob.png')
        # cv2.imwrite(prob_save_path, prob_uint8)

        # 2) save binary mask
        bin_mask = (prob_map >= threshold).astype(np.uint8) * 255
        bin_save_path = os.path.join(mask_out_dir, f'{name_stem}_bin.png')
        cv2.imwrite(bin_save_path, bin_mask)

        # 3) save gradcam (apply color map)
        cam_uint8 = (cam_map * 255.0).astype(np.uint8)
        cam_color = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)  # BGR
        # Option A: save heatmap only
        cam_save_path = os.path.join(grad_out_dir, f'{name_stem}_grad.png')
        cv2.imwrite(cam_save_path, cam_color)

        # # Option B: overlay on grayscale image for better visualization
        # overlay = cv2.cvtColor(img_disp, cv2.COLOR_GRAY2BGR)
        # overlay = cv2.resize(overlay, (cam_color.shape[1], cam_color.shape[0]), interpolation=cv2.INTER_LINEAR)
        # overlay_vis = cv2.addWeighted(overlay, 0.5, cam_color, 0.5, 0)
        # overlay_save_path = os.path.join(grad_out_dir, f'{name_stem}_overlay.png')
        # cv2.imwrite(overlay_save_path, overlay_vis)

        print(f'Processed {name} -> bin:{Path(bin_save_path).name}, grad:{Path(cam_save_path).name}')

# -----------------------
# Example usage (customize paths)
# -----------------------
input_dir = './archive/brisc2025/classification_test/meningioma/images'
mask_out_dir = './archive/brisc2025/classification_test/meningioma/masks'
grad_out_dir = './archive/brisc2025/classification_test/meningioma/gradcams'

# If your model expects fixed size (e.g., 224x224) set resize_to=(224,224), else None
save_mask_and_gradcam_from_dir(input_dir, mask_out_dir, grad_out_dir, model, grad_cam, resize_to=None, threshold=0.5)

Processed brisc2025_test_00255_me_ax_t1.jpg -> bin:brisc2025_test_00255_me_ax_t1_bin.png, grad:brisc2025_test_00255_me_ax_t1_grad.png
Processed brisc2025_test_00256_me_ax_t1.jpg -> bin:brisc2025_test_00256_me_ax_t1_bin.png, grad:brisc2025_test_00256_me_ax_t1_grad.png
Processed brisc2025_test_00257_me_ax_t1.jpg -> bin:brisc2025_test_00257_me_ax_t1_bin.png, grad:brisc2025_test_00257_me_ax_t1_grad.png
Processed brisc2025_test_00258_me_ax_t1.jpg -> bin:brisc2025_test_00258_me_ax_t1_bin.png, grad:brisc2025_test_00258_me_ax_t1_grad.png
Processed brisc2025_test_00259_me_ax_t1.jpg -> bin:brisc2025_test_00259_me_ax_t1_bin.png, grad:brisc2025_test_00259_me_ax_t1_grad.png
Processed brisc2025_test_00260_me_ax_t1.jpg -> bin:brisc2025_test_00260_me_ax_t1_bin.png, grad:brisc2025_test_00260_me_ax_t1_grad.png
Processed brisc2025_test_00261_me_ax_t1.jpg -> bin:brisc2025_test_00261_me_ax_t1_bin.png, grad:brisc2025_test_00261_me_ax_t1_grad.png
Processed brisc2025_test_00262_me_ax_t1.jpg -> bin:brisc2025_t

In [42]:

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def save_mask_and_gradcam_from_dir(input_dir,
                                   mask_out_dir,
                                   grad_out_dir,
                                   model,
                                   gradcam,
                                   img_exts=('.jpg', '.jpeg', '.png'),
                                   threshold=0.5,
                                   resize_to=None):
    """
    Processes every image file in input_dir (matching img_exts).
    - saves segmentation probability map and binary mask to mask_out_dir
    - saves Grad-CAM heatmap (colored) to grad_out_dir
    - resize_to: (W,H) or None to keep original size
    """
    ensure_dir(mask_out_dir)
    ensure_dir(grad_out_dir)

    image_paths = []
    for ext in img_exts:
        image_paths += glob(os.path.join(input_dir, f'*{ext}'))
    image_paths = sorted(image_paths)
    if len(image_paths) == 0:
        print('No images found in', input_dir)
        return


    device = next(model.parameters()).device

    for p in image_paths:
        name = Path(p).name
        name_stem = Path(p).stem

        # --- load and preprocess similarly to your test code ---
        pil = Image.open(p).convert('L')
        img_np = np.array(pil).astype(np.float32) / 255.0  # H x W

        # optional resize here to the model's expected input if needed:
        if resize_to is not None:
            img_disp = cv2.resize((img_np * 255).astype(np.uint8), resize_to, interpolation=cv2.INTER_LINEAR)
            img_for_model = cv2.resize(img_np, resize_to, interpolation=cv2.INTER_LINEAR)
        else:
            img_disp = (img_np * 255).astype(np.uint8)
            img_for_model = img_np

        # Apply same PerImageZScore transform you used
        # Using the simple transform you already wrote:
        transformed = PerImageZScore(p=1.0).apply(img_for_model)
        x = torch.from_numpy(transformed).unsqueeze(0).unsqueeze(0).float().to(device)  # (1,1,H,W)

        # --- Run model forward to get segmentation output (prob map) ---
        with torch.set_grad_enabled(True):   # Grad-CAM needs gradients; keep grads enabled
            out = model(x)   # raw logits (1,1,H',W') or (1,1,H,W) depending on model
            # If model output size != input size, interpolate to input size
            out_up = F.interpolate(out, size=img_for_model.shape, mode='bilinear', align_corners=False)
            prob_map = torch.sigmoid(out_up).squeeze().cpu().detach().numpy()  # H x W, float [0,1]

            # --- Grad-CAM ---
            cam_map, _ = gradcam(x, mode='sum')   # cam_map already resized inside gradcam to x.shape[2:]
            # cam_map is numpy HxW (float). Normalize 0..1
            cam_map = cam_map - cam_map.min()
            if cam_map.max() > 0:
                cam_map = cam_map / cam_map.max()
            cam_map = cv2.resize(cam_map, (prob_map.shape[1], prob_map.shape[0]), interpolation=cv2.INTER_LINEAR)

        # --- prepare saves ---
        # # 1) save probability map as 8-bit grayscale
        # prob_uint8 = (np.clip(prob_map, 0.0, 1.0) * 255.0).astype(np.uint8)
        # prob_save_path = os.path.join(mask_out_dir, f'{name_stem}_prob.png')
        # cv2.imwrite(prob_save_path, prob_uint8)

        # 2) save binary mask
        bin_mask = (prob_map >= threshold).astype(np.uint8) * 255
        bin_save_path = os.path.join(mask_out_dir, f'{name_stem}_bin.png')
        cv2.imwrite(bin_save_path, bin_mask)

        # 3) save gradcam (apply color map)
        cam_uint8 = (cam_map * 255.0).astype(np.uint8)
        cam_color = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)  # BGR
        # Option A: save heatmap only
        cam_save_path = os.path.join(grad_out_dir, f'{name_stem}_grad.png')
        cv2.imwrite(cam_save_path, cam_color)

        # # Option B: overlay on grayscale image for better visualization
        # overlay = cv2.cvtColor(img_disp, cv2.COLOR_GRAY2BGR)
        # overlay = cv2.resize(overlay, (cam_color.shape[1], cam_color.shape[0]), interpolation=cv2.INTER_LINEAR)
        # overlay_vis = cv2.addWeighted(overlay, 0.5, cam_color, 0.5, 0)
        # overlay_save_path = os.path.join(grad_out_dir, f'{name_stem}_overlay.png')
        # cv2.imwrite(overlay_save_path, overlay_vis)

        print(f'Processed {name} -> bin:{Path(bin_save_path).name}, grad:{Path(cam_save_path).name}')

# -----------------------
# Example usage (customize paths)
# -----------------------
input_dir = './archive/brisc2025/classification_test/no_tumor/images'
mask_out_dir = './archive/brisc2025/classification_test/no_tumor/masks'
grad_out_dir = './archive/brisc2025/classification_test/no_tumor/gradcams'

# If your model expects fixed size (e.g., 224x224) set resize_to=(224,224), else None
save_mask_and_gradcam_from_dir(input_dir, mask_out_dir, grad_out_dir, model, grad_cam, resize_to=None, threshold=0.5)

Processed brisc2025_test_00561_no_ax_t1.jpg -> bin:brisc2025_test_00561_no_ax_t1_bin.png, grad:brisc2025_test_00561_no_ax_t1_grad.png
Processed brisc2025_test_00562_no_ax_t1.jpg -> bin:brisc2025_test_00562_no_ax_t1_bin.png, grad:brisc2025_test_00562_no_ax_t1_grad.png
Processed brisc2025_test_00563_no_ax_t1.jpg -> bin:brisc2025_test_00563_no_ax_t1_bin.png, grad:brisc2025_test_00563_no_ax_t1_grad.png
Processed brisc2025_test_00564_no_ax_t1.jpg -> bin:brisc2025_test_00564_no_ax_t1_bin.png, grad:brisc2025_test_00564_no_ax_t1_grad.png
Processed brisc2025_test_00565_no_ax_t1.jpg -> bin:brisc2025_test_00565_no_ax_t1_bin.png, grad:brisc2025_test_00565_no_ax_t1_grad.png
Processed brisc2025_test_00566_no_ax_t1.jpg -> bin:brisc2025_test_00566_no_ax_t1_bin.png, grad:brisc2025_test_00566_no_ax_t1_grad.png
Processed brisc2025_test_00567_no_ax_t1.jpg -> bin:brisc2025_test_00567_no_ax_t1_bin.png, grad:brisc2025_test_00567_no_ax_t1_grad.png
Processed brisc2025_test_00568_no_ax_t1.jpg -> bin:brisc2025_t

In [43]:

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def save_mask_and_gradcam_from_dir(input_dir,
                                   mask_out_dir,
                                   grad_out_dir,
                                   model,
                                   gradcam,
                                   img_exts=('.jpg', '.jpeg', '.png'),
                                   threshold=0.5,
                                   resize_to=None):
    """
    Processes every image file in input_dir (matching img_exts).
    - saves segmentation probability map and binary mask to mask_out_dir
    - saves Grad-CAM heatmap (colored) to grad_out_dir
    - resize_to: (W,H) or None to keep original size
    """
    ensure_dir(mask_out_dir)
    ensure_dir(grad_out_dir)

    image_paths = []
    for ext in img_exts:
        image_paths += glob(os.path.join(input_dir, f'*{ext}'))
    image_paths = sorted(image_paths)
    if len(image_paths) == 0:
        print('No images found in', input_dir)
        return


    device = next(model.parameters()).device

    for p in image_paths:
        name = Path(p).name
        name_stem = Path(p).stem

        # --- load and preprocess similarly to your test code ---
        pil = Image.open(p).convert('L')
        img_np = np.array(pil).astype(np.float32) / 255.0  # H x W

        # optional resize here to the model's expected input if needed:
        if resize_to is not None:
            img_disp = cv2.resize((img_np * 255).astype(np.uint8), resize_to, interpolation=cv2.INTER_LINEAR)
            img_for_model = cv2.resize(img_np, resize_to, interpolation=cv2.INTER_LINEAR)
        else:
            img_disp = (img_np * 255).astype(np.uint8)
            img_for_model = img_np

        # Apply same PerImageZScore transform you used
        # Using the simple transform you already wrote:
        transformed = PerImageZScore(p=1.0).apply(img_for_model)
        x = torch.from_numpy(transformed).unsqueeze(0).unsqueeze(0).float().to(device)  # (1,1,H,W)

        # --- Run model forward to get segmentation output (prob map) ---
        with torch.set_grad_enabled(True):   # Grad-CAM needs gradients; keep grads enabled
            out = model(x)   # raw logits (1,1,H',W') or (1,1,H,W) depending on model
            # If model output size != input size, interpolate to input size
            out_up = F.interpolate(out, size=img_for_model.shape, mode='bilinear', align_corners=False)
            prob_map = torch.sigmoid(out_up).squeeze().cpu().detach().numpy()  # H x W, float [0,1]

            # --- Grad-CAM ---
            cam_map, _ = gradcam(x, mode='sum')   # cam_map already resized inside gradcam to x.shape[2:]
            # cam_map is numpy HxW (float). Normalize 0..1
            cam_map = cam_map - cam_map.min()
            if cam_map.max() > 0:
                cam_map = cam_map / cam_map.max()
            cam_map = cv2.resize(cam_map, (prob_map.shape[1], prob_map.shape[0]), interpolation=cv2.INTER_LINEAR)

        # --- prepare saves ---
        # # 1) save probability map as 8-bit grayscale
        # prob_uint8 = (np.clip(prob_map, 0.0, 1.0) * 255.0).astype(np.uint8)
        # prob_save_path = os.path.join(mask_out_dir, f'{name_stem}_prob.png')
        # cv2.imwrite(prob_save_path, prob_uint8)

        # 2) save binary mask
        bin_mask = (prob_map >= threshold).astype(np.uint8) * 255
        bin_save_path = os.path.join(mask_out_dir, f'{name_stem}_bin.png')
        cv2.imwrite(bin_save_path, bin_mask)

        # 3) save gradcam (apply color map)
        cam_uint8 = (cam_map * 255.0).astype(np.uint8)
        cam_color = cv2.applyColorMap(cam_uint8, cv2.COLORMAP_JET)  # BGR
        # Option A: save heatmap only
        cam_save_path = os.path.join(grad_out_dir, f'{name_stem}_grad.png')
        cv2.imwrite(cam_save_path, cam_color)

        # # Option B: overlay on grayscale image for better visualization
        # overlay = cv2.cvtColor(img_disp, cv2.COLOR_GRAY2BGR)
        # overlay = cv2.resize(overlay, (cam_color.shape[1], cam_color.shape[0]), interpolation=cv2.INTER_LINEAR)
        # overlay_vis = cv2.addWeighted(overlay, 0.5, cam_color, 0.5, 0)
        # overlay_save_path = os.path.join(grad_out_dir, f'{name_stem}_overlay.png')
        # cv2.imwrite(overlay_save_path, overlay_vis)

        print(f'Processed {name} -> bin:{Path(bin_save_path).name}, grad:{Path(cam_save_path).name}')

# -----------------------
# Example usage (customize paths)
# -----------------------
input_dir = './archive/brisc2025/classification_test/pituitary/images'
mask_out_dir = './archive/brisc2025/classification_test/pituitary/masks'
grad_out_dir = './archive/brisc2025/classification_test/pituitary/gradcams'

# If your model expects fixed size (e.g., 224x224) set resize_to=(224,224), else None
save_mask_and_gradcam_from_dir(input_dir, mask_out_dir, grad_out_dir, model, grad_cam, resize_to=None, threshold=0.5)

Processed brisc2025_test_00701_pi_ax_t1.jpg -> bin:brisc2025_test_00701_pi_ax_t1_bin.png, grad:brisc2025_test_00701_pi_ax_t1_grad.png
Processed brisc2025_test_00702_pi_ax_t1.jpg -> bin:brisc2025_test_00702_pi_ax_t1_bin.png, grad:brisc2025_test_00702_pi_ax_t1_grad.png
Processed brisc2025_test_00703_pi_ax_t1.jpg -> bin:brisc2025_test_00703_pi_ax_t1_bin.png, grad:brisc2025_test_00703_pi_ax_t1_grad.png
Processed brisc2025_test_00704_pi_ax_t1.jpg -> bin:brisc2025_test_00704_pi_ax_t1_bin.png, grad:brisc2025_test_00704_pi_ax_t1_grad.png
Processed brisc2025_test_00705_pi_ax_t1.jpg -> bin:brisc2025_test_00705_pi_ax_t1_bin.png, grad:brisc2025_test_00705_pi_ax_t1_grad.png
Processed brisc2025_test_00706_pi_ax_t1.jpg -> bin:brisc2025_test_00706_pi_ax_t1_bin.png, grad:brisc2025_test_00706_pi_ax_t1_grad.png
Processed brisc2025_test_00707_pi_ax_t1.jpg -> bin:brisc2025_test_00707_pi_ax_t1_bin.png, grad:brisc2025_test_00707_pi_ax_t1_grad.png
Processed brisc2025_test_00708_pi_ax_t1.jpg -> bin:brisc2025_t

In [45]:
import os, csv
from pathlib import Path

# ---------- CHANGE THIS TO YOUR TEST FOLDER ----------
ROOT = Path("/content/archive/brisc2025/classification_test")
# ------------------------------------------------------

OUT_CSV = Path("/content/archive/brisc2025/test_labels.csv")
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)

# detect classes (folders)
classes = sorted([
    d.name for d in ROOT.iterdir()
    if d.is_dir() and not d.name.startswith(".")
])

print("Detected classes:", classes)

# map classes to integer labels
class_to_label = {cls: idx for idx, cls in enumerate(classes)}
print("Label map:", class_to_label)

def find_by_stem(folder, stem):
    """Find file with the same stem (with any extension)."""
    if not folder.exists():
        return ""
    for ext in [".png", ".jpg", ".jpeg", ".bmp", ".tif"]:
        f = folder / f"{stem}{ext}"
        if f.exists():
            return str(f.resolve())
    return ""

rows = []

# build rows for CSV
for cls in classes:
    img_dir = ROOT / cls / "images"
    mask_dir = ROOT / cls / "masks"
    gcam_dir = ROOT / cls / "gradcams"

    if not img_dir.exists():
        print(f"[WARN] No images folder for {cls}, skipping.")
        continue

    for f in sorted(img_dir.iterdir()):
        if f.suffix.lower() not in [".png", ".jpg", ".jpeg", ".bmp", ".tif"]:
            continue

        stem = f.stem

        img_path = str(f.resolve())
        mask_path = find_by_stem(mask_dir, stem)
        gcam_path = find_by_stem(gcam_dir, stem)

        rows.append([
            img_path,
            mask_path,
            gcam_path,
            class_to_label[cls],
            cls,
            "test"
        ])

print("Total test samples:", len(rows))

# write CSV
with open(OUT_CSV, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["image_path", "mask_path", "gradcam_path", "label", "class_name", "split"])
    w.writerows(rows)

print("Saved CSV to:", OUT_CSV)

Detected classes: ['glioma', 'meningioma', 'no_tumor', 'pituitary']
Label map: {'glioma': 0, 'meningioma': 1, 'no_tumor': 2, 'pituitary': 3}
Total test samples: 1000
Saved CSV to: /content/archive/brisc2025/test_labels.csv


In [48]:
# ===== Test evaluation: run on test_labels.csv and save metrics =====
# Paste & run in Colab

import torch, pandas as pd, numpy as np, os
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import cv2
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns, matplotlib.pyplot as plt
import torchvision.models as models
import torch.nn as nn

# ---------- CONFIG ----------
TEST_CSV = "/content/archive/brisc2025/test_labels.csv"   # path to test_labels.csv you just created
MODEL_PATH = "/content/archive/brisc2025/checkpoints/best_resnet18.pth"  # path to your saved best model
OUT_DIR = "/content/archive/brisc2025/checkpoints/classification_test_output"  # where to save confusion matrix and predictions
IMG_SIZE = 256
BATCH_SIZE = 32
NUM_WORKERS = 2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# ----------------------------

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# ---------- Dataset (3-channel: gray + mask + gradcam) ----------
class MaskGradcamTestDataset(Dataset):
    def __init__(self, df, img_size=IMG_SIZE, transforms=None):
        self.df = df.reset_index(drop=True)
        self.img_size = img_size
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def _read_gray_safe(self, p):
        if (not isinstance(p, str)) or (p == "") or (not Path(p).exists()):
            return np.zeros((self.img_size, self.img_size), dtype=np.float32)
        img = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
        if img is None:
            return np.zeros((self.img_size, self.img_size), dtype=np.float32)
        img = cv2.resize(img, (self.img_size, self.img_size), interpolation=cv2.INTER_LINEAR)
        return (img.astype(np.float32) / 255.0)

    def _read_mask_safe(self, p):
        if (not isinstance(p, str)) or (p == "") or (not Path(p).exists()):
            return np.zeros((self.img_size, self.img_size), dtype=np.float32)
        m = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
        if m is None:
            return np.zeros((self.img_size, self.img_size), dtype=np.float32)
        m = cv2.resize(m, (self.img_size, self.img_size), interpolation=cv2.INTER_NEAREST)
        if m.max() > 1:
            m = m.astype(np.float32) / 255.0
        else:
            m = m.astype(np.float32)
        return m

    def _read_gcam_safe(self, p):
        if (not isinstance(p, str)) or (p == "") or (not Path(p).exists()):
            return np.zeros((self.img_size, self.img_size), dtype=np.float32)
        g = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
        if g is None:
            return np.zeros((self.img_size, self.img_size), dtype=np.float32)
        g = cv2.resize(g, (self.img_size, self.img_size), interpolation=cv2.INTER_LINEAR)
        g = g.astype(np.float32)
        gmax = g.max()
        if gmax > 0:
            g = g / gmax
        return g

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img = self._read_gray_safe(row['image_path'])
        mask = self._read_mask_safe(row['mask_path'])
        gcam = self._read_gcam_safe(row['gradcam_path'])
        stacked = np.stack([img, mask, gcam], axis=2)  # H,W,3
        # convert to CHW tensor
        tensor = torch.from_numpy(stacked.transpose(2,0,1)).float()
        return tensor, int(row['label']), row['image_path']

# ---------- Model helper: build ResNet18 and adapt conv1 if needed ----------
def make_resnet18(num_classes, in_channels=3, pretrained=False):
    model = models.resnet18(pretrained=pretrained)
    if in_channels != 3:
        old = model.conv1
        new_conv = nn.Conv2d(in_channels, old.out_channels, kernel_size=old.kernel_size,
                             stride=old.stride, padding=old.padding, bias=(old.bias is not None))
        with torch.no_grad():
            old_w = old.weight.data
            new_w = torch.zeros((old_w.shape[0], in_channels, *old_w.shape[2:]), dtype=old_w.dtype)
            new_w[:, :3, :, :] = old_w
            if in_channels > 3:
                mean_w = old_w.mean(dim=1, keepdim=True)
                for c in range(3, in_channels):
                    new_w[:, c:c+1, :, :] = mean_w
            new_conv.weight.data = new_w
        model.conv1 = new_conv
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# ---------- Load test CSV ----------
df_test = pd.read_csv(TEST_CSV)
if 'split' in df_test.columns:
    df_test = df_test[df_test['split'].str.lower() == 'test'].reset_index(drop=True)

# build label->class mapping from CSV (ensure consistent)
label_to_class = {}
for _, r in df_test[['label','class_name']].drop_duplicates().iterrows():
    label_to_class[int(r['label'])] = r['class_name']
# create ordered class_names list by label index
num_labels = int(df_test['label'].max()) + 1
class_names = [label_to_class.get(i, str(i)) for i in range(num_labels)]

print("Test samples:", len(df_test))
print("Classes:", class_names)

# ---------- Dataset and loader ----------
test_ds = MaskGradcamTestDataset(df_test, img_size=IMG_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

# ---------- Load model ----------
device = DEVICE
print("Using device:", device)
model = make_resnet18(num_classes=num_labels, in_channels=3, pretrained=False).to(device)
if not Path(MODEL_PATH).exists():
    raise FileNotFoundError(f"Model not found at {MODEL_PATH}")
state = torch.load(MODEL_PATH, map_location=device)
# detect if full state dict or model_state_dict
if isinstance(state, dict) and 'model_state_dict' in state:
    model.load_state_dict(state['model_state_dict'])
else:
    # assume it's a plain state_dict
    model.load_state_dict(state)
model.eval()

# ---------- Inference over test set ----------
all_preds = []
all_probs = []
all_labels = []
all_paths = []

with torch.no_grad():
    for imgs, labels, paths in test_loader:
        imgs = imgs.to(device).float()
        out = model(imgs)
        probs = F.softmax(out, dim=1).cpu().numpy()
        preds = probs.argmax(axis=1)
        all_preds.extend(preds.tolist())
        all_probs.extend(probs.tolist())
        all_labels.extend(labels.numpy().tolist())
        all_paths.extend(list(paths))

# ---------- Metrics ----------
acc = accuracy_score(all_labels, all_preds)
print(f"\nTest Accuracy: {acc:.4f}\n")

print("Classification report:\n")
print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))

cm = confusion_matrix(all_labels, all_preds)
print("Confusion matrix:\n", cm)

# ---------- Save confusion matrix heatmap ----------
cm_path = Path(OUT_DIR) / "confusion_matrix_test.png"
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - Test")
plt.tight_layout()
plt.savefig(str(cm_path), dpi=150)
plt.close()
print("Saved confusion matrix to:", cm_path)

# ---------- Save per-image predictions CSV ----------
rows = []
for p, true, pred, prob in zip(all_paths, all_labels, all_preds, all_probs):
    rows.append({
        'image_path': p,
        'true_label': int(true),
        'true_class': label_to_class.get(int(true), str(true)),
        'pred_label': int(pred),
        'pred_class': label_to_class.get(int(pred), str(pred)),
        'pred_confidence': float(prob[int(pred)])
    })
out_csv = Path(OUT_DIR) / "test_predictions.csv"
pd.DataFrame(rows).to_csv(out_csv, index=False)
print("Saved test predictions CSV to:", out_csv)

# ---------- Summary ----------
print("\nDONE. Summary:")
print(f"Test samples: {len(df_test)}")
print(f"Saved confusion matrix: {cm_path}")
print(f"Saved per-sample predictions: {out_csv}")


Test samples: 1000
Classes: ['glioma', 'meningioma', 'no_tumor', 'pituitary']
Using device: cuda


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



Test Accuracy: 0.9860

Classification report:

              precision    recall  f1-score   support

      glioma     1.0000    0.9724    0.9860       254
  meningioma     0.9805    0.9837    0.9821       306
    no_tumor     0.9929    1.0000    0.9964       140
   pituitary     0.9770    0.9933    0.9851       300

    accuracy                         0.9860      1000
   macro avg     0.9876    0.9874    0.9874      1000
weighted avg     0.9861    0.9860    0.9860      1000

Confusion matrix:
 [[247   4   0   3]
 [  0 301   1   4]
 [  0   0 140   0]
 [  0   2   0 298]]
Saved confusion matrix to: /content/archive/brisc2025/checkpoints/classification_test_output/confusion_matrix_test.png
Saved test predictions CSV to: /content/archive/brisc2025/checkpoints/classification_test_output/test_predictions.csv

DONE. Summary:
Test samples: 1000
Saved confusion matrix: /content/archive/brisc2025/checkpoints/classification_test_output/confusion_matrix_test.png
Saved per-sample predictions: /co